# Deepface Face Verification test 

### Import relevant libraries:

In [15]:
# !pip install deepface
from deepface import DeepFace

import matplotlib.pyplot as plt
import pandas as pd                 
import regex as re

import os

# change plot output to white
params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}
plt.rcParams.update(params)

result_df = pd.read_csv('./result/result_debface.csv')
result_benchmark_gender = pd.read_csv('./result/deepface_benchmark_gender.csv')
benchmark_df = pd.read_csv('./data/LFW-csv/pairs.csv')

In [16]:
result_df

,Model,Dataset,CM_ACC,Precision,Recall,Total Images
0,VGG-Face,LFW,88.0,100.0,88.0,9164
1,Facenet,LFW,65.0,100.0,65.0,9164
2,OpenFace,LFW,1.0,100.0,1.0,9164
3,DeepFace,LFW,26.0,100.0,26.0,9164
4,ArcFace,LFW,83.0,100.0,83.0,9164
5,VGG-Face,LFW-original,85.0,100.0,85.0,6000
6,Facenet,LFW-original,61.0,100.0,61.0,6000
7,OpenFace,LFW-original,2.0,100.0,2.0,6000
8,DeepFace,LFW-original,27.0,100.0,27.0,6000
9,OpenFace,LFW,15.0,100.0,15.0,6000


In [17]:
result_benchmark_gender

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender
0,Facenet,LFW_gender,79.0,100.0,58.0,4489,Male
1,Facenet,LFW_gender,78.0,99.0,57.0,1511,Female
2,VGG-Face,LFW_gender,92.0,96.0,88.0,1511,Female
3,VGG-Face,LFW_gender,84.0,77.0,95.0,4489,Male
4,OpenFace,LFW_gender,59.0,88.0,23.0,1511,Female
5,OpenFace,LFW_gender,55.0,85.0,12.0,4489,Male
6,DeepFace,LFW_gender,61.0,79.0,33.0,1511,Female
7,DeepFace,LFW_gender,65.0,76.0,44.0,4489,Male


# Deepface Threshold results:

In [4]:
## Results based on https://youtu.be/i_MOwvhbLdI 
_vgg_face = {"Model": "VGGFace", "Accuracy": 89.28, "Precision": 97.41, "Recall": 80.71}
_facenet = {"Model": "FaceNet", "Accuracy": 98.21, "Precision": 100, "Recall": 96.42}
_openface = {"Model": "OpenFace", "Accuracy": 57.85, "Precision": 95.83, "Recall": 16.42}
_deepface = {"Model": "DeepFace", "Accuracy": 54.64, "Precision": 100, "Recall": 9.28}

models_results = [_facenet, _openface, _deepface]

result_model_df = pd.DataFrame([_vgg_face])
for model in models_results:
    result_model = pd.DataFrame([model])
    frames = [result_model_df, result_model]
    result_model_df = pd.concat(frames)

    result_model_df = result_model_df.reset_index()
    result_model_df.drop(columns=['index'], inplace=True)

result_model_df

,Model,Accuracy,Precision,Recall
0,VGGFace,89.28,97.41,80.71
1,FaceNet,98.21,100.00,96.42
2,OpenFace,57.85,95.83,16.42
3,DeepFace,54.64,100.00,9.28


#### Paramater settings:

In [5]:
# select a DATASET from data folder:
# LFW, LFW-original
DATASET = "LFW"

# test with gender split LFW
DATASET_LFW_SPLIT = "LFW_gender"

# select a model to run:
# models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
MODEL = "DeepFace"

# Faces are represented as vectors, a face pair of same person should be similar, calculated by different metrics
# metrics = ["cosine", "euclidean", "euclidean_l2"]
METRIC = "euclidean_l2"

# Face detector, backend that helps aligning the face for better accuracy
# backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']
BACKEND = ['opencv', 'retinaface', 'mtcnn']

# Select 'Male' or 'Female'
TEST_GENDER = 'Male'

### Preset the data path directory

In [6]:
# Test additional data directory:
# dir_path = f"./additional-data/{DATASET_TEST}"

# Test dataset directory:
dir_path = f"./data/{DATASET_LFW_SPLIT}"

### Function to perform test with Deepface

In [7]:
def deepface_get_accuracy(img_dir_path, model_used):

    # store the results of each verification:
    tp, tn, fp, fn = 0, 0, 0, 0
    image_files = os.listdir(img_dir_path)
    truth_images = []    
    false_images = []    
    
    # If the directory has less than 2 images there is no need to test face recognition
    if len(image_files) < 2:
        return None

    else:
        # select the first image 
        image_1 = f"{img_dir_path}/{image_files[0]}"

        # loop through the other images and verify if they are the same
        for img in image_files[1:]:
            try:
                match = re.search(f"_\d.*\.jpg", str(img))
                # Find all annotation of the files, replace <NAME> with the desired person to test
                # get image name to replace the match string with empty string
                # NAME_\d\.jpg
                # NAME_\d.*\.jpg
                NAME = img.replace(match.group(0), "")
                
            except AttributeError:
                print("IMAGE: " + image_1)
                exit(1)

            
            flag = None
            if re.search(f"{NAME}_\d.*\.jpg", str(img)):
                truth_images.append(img)
                flag = True
            else:
                false_images.append(img)    
                flag = False

            try:
                img_path = f"{img_dir_path}/{img}"
                verification = DeepFace.verify(img1_path = image_1, 
                                                img2_path = img_path, 
                                                model_name=model_used, 
                                                enforce_detection=False,
                                                distance_metric= METRIC, 
                                                detector_backend = BACKEND[0])
                                                
                verification_res = verification["verified"]

                # Rerun the verification with backend Retinaface if the result is not reliable with OpenCV
                if not verification_res:
                    verification = DeepFace.verify(img1_path = image_1, 
                                                img2_path = img_path, 
                                                model_name=model_used, 
                                                enforce_detection=False,
                                                distance_metric= METRIC, 
                                                detector_backend = BACKEND[1])
                    verification_res = verification["verified"]
                # print(f"image_1: {image_1}, image: {img_path}, predicted: {verification_res}, flag:{flag}")
                
                # Truth Positive if flag and predicted are True:
                if flag and verification_res:
                    tp += 1
                    
                # Truth Negative if flag and predicted are False:
                elif flag == False and verification_res == False:
                    tn += 1

                # False Positive if flag is False and predicted is True:
                elif flag == False and verification_res == True:
                    fp += 1

                # False Negative if flag is True and predicted is False:
                elif flag == True and verification_res == False:
                    fn += 1        

            except Exception as e:
                print(e)

        # confusion matrix
        cm_acc = round((tp + tn) / (tp + tn + fp + fn), 2) * 100

        return {"cm_acc": cm_acc, "tp": tp, "tn":tn, "fp":fp, "fn":fn, "image_files": len(image_files)-1}


### Deepface Dataset testing:

In [8]:
def deepface_run_dataset(dataset, dataset_path, MODEL):

    avg_truth_pos = 0
    avg_truth_neg = 0
    avg_false_pos = 0
    avg_false_neg = 0
    total_imgs = 0
    
    iteration = 0

    people = os.listdir(dataset_path)
    for plp in people:

        img_path = f"{dataset_path}/{plp}"
        data_res = deepface_get_accuracy(img_path, MODEL)

        if data_res != None:
            avg_truth_pos += data_res["tp"]
            avg_truth_neg += data_res["tn"]
            avg_false_pos += data_res["fp"]
            avg_false_neg += data_res["fn"]
            total_imgs += data_res["image_files"]

            iteration += 1
            if iteration == 10:
                print(img_path)
                break

    # calculate confusion matrix:
    # accuracy:
    cm_acc = round((avg_truth_pos + avg_truth_neg) / (avg_truth_pos + avg_truth_neg + avg_false_pos + avg_false_neg), 2) * 100
    # precision:
    cm_pre = round( (avg_truth_pos) / (avg_truth_pos + avg_false_pos), 2) * 100
    # recall: 
    cm_rec = round( (avg_truth_pos) / (avg_truth_pos + avg_false_neg), 2) * 100


    return {"Model": MODEL, "Dataset":dataset, "CM_ACC": cm_acc, "Precision":cm_pre, "Recall":cm_rec,"Total Images": total_imgs},\
         {"TP": avg_truth_pos, "TN":avg_truth_neg, "FP":avg_false_pos, "FN":avg_false_neg}

## Face recognition Benchmark Test:
- using LFW `pairs.csv` 

In [9]:
def deepface_benchmark(dataset, dataset_path, benchmark_df, model_used):

    tp, tn, fp, fn = 0, 0, 0, 0
    iteration = 1
    match_case = True
    for index, row in benchmark_df.iterrows():

        if iteration > 300 and match_case:
            match_case = False
            iteration = 1
        elif iteration > 300 and not match_case:
            match_case = True
            iteration = 1

        if match_case:
            img_name = row["name"]
            img_num_1 = f'{row["imagenum1"]:04d}'
            img_num_2 = f'{int(row["imagenum2"]):04d}'

            img_path_1 = f'{dataset_path}/{img_name}/{img_name}_{img_num_1}.jpg'
            img_path_2 = f'{dataset_path}/{img_name}/{img_name}_{img_num_2}.jpg'
            iteration += 1
        
        elif match_case == False:
            img_name_a = row["name"]
            img_num_1 = f'{row["imagenum1"]:04d}'
            img_name_b = f'{row["imagenum2"]}'
            img_num_2 = f'{int(row[3]):04d}'

            img_path_1 = f'{dataset_path}/{img_name}/{img_name_a}_{img_num_1}.jpg'
            img_path_2 = f'{dataset_path}/{img_name}/{img_name_b}_{img_num_2}.jpg'
            iteration += 1    

        try:
            # verification = DeepFace.verify(img1_path = img_path_1, img2_path = img_path_2, model_name=MODEL, enforce_detection=False)
            verification = DeepFace.verify(img1_path = img_path_1, 
                                            img2_path = img_path_2, 
                                            model_name=model_used, 
                                            enforce_detection=False,
                                            distance_metric= METRIC, 
                                            detector_backend = BACKEND[0])
                                            
            verification_res = verification["verified"]

            # Rerun the verification with backend RetinaFace if the result is not reliable with OpenCV
            if not verification_res:
                verification = DeepFace.verify(img1_path = img_path_1, 
                                            img2_path = img_path_2, 
                                            model_name=model_used, 
                                            enforce_detection=False,
                                            distance_metric= METRIC, 
                                            detector_backend = BACKEND[1])
                verification_res = verification["verified"]
            
            # Truth Positive if flag and predicted are True:
            if match_case and verification_res:
                tp += 1
            # Truth Negative if flag and predicted are False:
            elif match_case == False and verification_res == False:
                tn += 1

            # False Positive if flag is False and predicted is True:
            elif match_case == False and verification_res == True:
                fp += 1

            # False Negative if flag is True and predicted is False:
            elif match_case == True and verification_res == False:
                fn += 1        

        except Exception as e:
            print(e)

    # calculate confusion matrix:
    cm_acc = round((tp + tn) / (tp + tn + fp + fn), 2) * 100
    # precision:
    cm_pre = round( (tp) / (tp + fp), 2) * 100
    # recall: 
    cm_rec = round( (tp) / (tp + fn), 2) * 100


    return {"Model": MODEL, "Dataset":dataset, "CM_ACC": cm_acc, "Precision":cm_pre, "Recall":cm_rec,"Total Images": index+1},\
         {"TP": tp, "TN":tn, "FP":fp, "FN":fn}

### Benchmark testing with LFW gender split

In [10]:
def find_image(dataset_path, img_name, img_num):
    pathA = f'{dataset_path}/Female/{img_name}/{img_name}_{img_num}.jpg'
    pathB = f'{dataset_path}/Male/{img_name}/{img_name}_{img_num}.jpg'
    path_truth = None
    gender = None
    
    if os.path.exists(pathA):
        path_truth = pathA
        gender = 'Female'
    else:
        path_truth = pathB
        gender = 'Male'

    return path_truth, gender


def deepface_benchmark_lfw_split(dataset, dataset_path, benchmark_df, model_used, test_gender):

    tp, tn, fp, fn = 0, 0, 0, 0
    tested = 0
    iteration = 1
    match_case = True
    for index, row in benchmark_df.iterrows():

        if iteration > 300 and match_case:
            match_case = False
            iteration = 1
        elif iteration > 300 and not match_case:
            match_case = True
            iteration = 1

        if match_case:
            img_name = row["name"]
            img_num_1 = f'{row["imagenum1"]:04d}'
            img_num_2 = f'{int(row["imagenum2"]):04d}'

            img_path_1, gender_1 = find_image(dataset_path, img_name, img_num_1)
            img_path_2, gender_2 = find_image(dataset_path, img_name, img_num_2)
            iteration += 1
        
        elif match_case == False:
            img_name_a = row["name"]
            img_num_1 = f'{row["imagenum1"]:04d}'
            img_name_b = f'{row["imagenum2"]}'
            img_num_2 = f'{int(row[3]):04d}'

            img_path_1, gender_1 = find_image(dataset_path, img_name_a, img_num_1)
            img_path_2, gender_2 = find_image(dataset_path, img_name_b, img_num_2)
            iteration += 1    

        if test_gender == gender_1:
            try:
                # verification = DeepFace.verify(img1_path = img_path_1, img2_path = img_path_2, model_name=MODEL, enforce_detection=False)
                verification = DeepFace.verify(img1_path = img_path_1, 
                                                img2_path = img_path_2, 
                                                model_name=model_used, 
                                                enforce_detection=False,
                                                distance_metric= METRIC, 
                                                detector_backend = BACKEND[0])
                                                
                verification_res = verification["verified"]

                # Rerun the verification with backend RetinaFace if the result is not reliable with OpenCV
                if not verification_res:
                    verification = DeepFace.verify(img1_path = img_path_1, 
                                                img2_path = img_path_2, 
                                                model_name=model_used, 
                                                enforce_detection=False,
                                                distance_metric= METRIC, 
                                                detector_backend = BACKEND[1])
                    verification_res = verification["verified"]
                
                # Truth Positive if flag and predicted are True:
                if match_case and verification_res:
                    tp += 1
                # Truth Negative if flag and predicted are False:
                elif match_case == False and verification_res == False:
                    tn += 1

                # False Positive if flag is False and predicted is True:
                elif match_case == False and verification_res == True:
                    fp += 1

                # False Negative if flag is True and predicted is False:
                elif match_case == True and verification_res == False:
                    fn += 1
                
                # one image tested with the demanded gender 
                tested += 1

            except Exception as e:
                print(e)
        else:
            pass

    # calculate confusion matrix:
    cm_acc = round((tp + tn) / (tp + tn + fp + fn), 2) * 100
    # precision:
    cm_pre = round( (tp) / (tp + fp), 2) * 100
    # recall: 
    cm_rec = round( (tp) / (tp + fn), 2) * 100


    return {"Model": MODEL, "Dataset":dataset, "CM_ACC": cm_acc, "Precision":cm_pre, "Recall":cm_rec,"Total Images": tested, "Gender": test_gender},\
         {"TP": tp, "TN":tn, "FP":fp, "FN":fn}

### Execution of the Deepface test:

In [ ]:
# random small testing
deepface_res = deepface_get_accuracy(dir_name, MODEL)

In [11]:
# manual testing
deepface_res_dataset, cm_attr = deepface_run_dataset(DATASET, dir_path, MODEL)

1/1 [==============================] - 0s 23ms/step
./data/LFW/Abid_Hamid_Mahmud_Al-Tikriti


In [33]:
# benchmark testing
deepface_benchmark_res, cm_benchmark = deepface_benchmark(DATASET, dir_path, benchmark_df, MODEL)

1/1 [==============================] - 0s 42ms/step
('Confirm that ', './data/LFW/Zico/Abdel_Madi_Shabneh_0001.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Abdel_Madi_Shabneh_0001.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Abdel_Madi_Shabneh_0001.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Abdul_Rahman_0001.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Abel_Pacheco_0001.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Abel_Pacheco_0002.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Afton_Smith_0001.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Ahmad_Jbarah_0001.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Akhmed_Zakayev_0002.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Alan_Dershowitz_0001.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Alanis_Morissette_0001.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Alexander_Lukashenko_0001.jpg', ' exists')
('Confirm that ', './data/LFW/Zico/Alfonso_Cuaron_0001.jpg', ' exists')
(

In [11]:
# benchmark testing LFW with gender split 
deepface_benchmark_res, cm_benchmark = deepface_benchmark_lfw_split(DATASET_LFW_SPLIT, dir_path, benchmark_df, MODEL, TEST_GENDER)

1/1 [==============================] - 0s 100ms/step


## Result of the Deepface test:

### Small test result:

In [ ]:
cm_acc = deepface_res["cm_acc"]
truth_pos = deepface_res["tp"]
truth_neg = deepface_res["tn"]
false_pos = deepface_res["fp"]
false_neg = deepface_res["fn"]    

total_imgs = deepface_res["image_files"]

print(f"\
Accuracy of {MODEL} is {cm_acc}% out of {total_imgs} images from {DATASET} dataset.\n\
===================\n\
Truth Positive: {truth_pos} \n\
Truth Negative: {truth_neg} \n\
False Positive: {false_pos} \n\
False Negative: {false_neg} \n\
===================")

### Manual Testing:

In [72]:
# confusion matrix accuracy:
cm_acc = deepface_res_dataset["CM_ACC"]
cm_pre = deepface_res_dataset["Precision"]
cm_rec = deepface_res_dataset["Recall"]
truth_pos = cm_attr["TP"]
truth_neg = cm_attr["TN"]
false_pos = cm_attr["FP"]
false_neg = cm_attr["FN"]

# total images
total_imgs = deepface_res_dataset["Total Images"]

print(f"\
Accuracy of {MODEL} is {cm_acc}% out of {total_imgs} images from {DATASET} dataset.\n\
The Precision is {cm_pre} and Recall is {cm_rec}\n\
===================\n\
Truth Positive: {truth_pos} \n\
Truth Negative: {truth_neg} \n\
False Positive: {false_pos} \n\
False Negative: {false_neg} \n\
===================")

NameError: name 'deepface_res_dataset' is not defined

### Benchmark testing:

In [12]:
# confusion matrix accuracy:
cm_acc = deepface_benchmark_res["CM_ACC"]
cm_pre = deepface_benchmark_res["Precision"]
cm_rec = deepface_benchmark_res["Recall"]
truth_pos = cm_benchmark["TP"]
truth_neg = cm_benchmark["TN"]
false_pos = cm_benchmark["FP"]
false_neg = cm_benchmark["FN"]

# total images
total_imgs = deepface_benchmark_res["Total Images"]

print(f"\
Accuracy of {MODEL} is {cm_acc}% out of {total_imgs} images from {DATASET} dataset.\n\
The Precision is {cm_pre} and Recall is {cm_rec}\n\
===================\n\
Truth Positive: {truth_pos} \n\
Truth Negative: {truth_neg} \n\
False Positive: {false_pos} \n\
False Negative: {false_neg} \n\
===================")

Accuracy of DeepFace is 65.0% out of 4489 images from LFW dataset.
The Precision is 76.0 and Recall is 44.0
Truth Positive: 973 
Truth Negative: 1954 
False Positive: 312 
False Negative: 1250 


# Saving result:

#### Write to CSV for Manual + Benchmark Testing

In [52]:
result_df_new = pd.DataFrame([deepface_benchmark_res])
# result_df_new = pd.DataFrame([deepface_res_dataset])

frames = [result_df, result_df_new]
result_df = pd.concat(frames)

result_df = result_df.reset_index()
result_df.drop(columns=['index'], inplace=True)
result_df

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender
0,LFW,88.0,100.0,88.0,9164.0,NaN,NaN
1,LFW,65.0,100.0,65.0,9164.0,NaN,NaN
2,LFW,1.0,100.0,1.0,9164.0,NaN,NaN
3,LFW,26.0,100.0,26.0,9164.0,NaN,NaN
4,LFW,83.0,100.0,83.0,9164.0,NaN,NaN
5,LFW-original,85.0,100.0,85.0,6000.0,NaN,NaN
6,LFW-original,61.0,100.0,61.0,6000.0,NaN,NaN
7,LFW-original,2.0,100.0,2.0,6000.0,NaN,NaN
8,LFW-original,27.0,100.0,27.0,6000.0,NaN,NaN
9,LFW,15.0,100.0,15.0,6000.0,NaN,NaN


In [28]:
# Write results to csv file
result_df.to_csv("./result/result_debface.csv", index=False)

#### Write to CSV for Benchmark LFW gender split

In [13]:
result_df_new = pd.DataFrame([deepface_benchmark_res])

frames = [result_benchmark_gender, result_df_new]
result_benchmark_df = pd.concat(frames)

result_benchmark_df = result_benchmark_df.reset_index()
result_benchmark_df.drop(columns=['index'], inplace=True)
result_benchmark_df

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender
0,Facenet,LFW_gender,79.0,100.0,58.0,4489,Male
1,Facenet,LFW_gender,78.0,99.0,57.0,1511,Female
2,VGG-Face,LFW_gender,92.0,96.0,88.0,1511,Female
3,VGG-Face,LFW_gender,84.0,77.0,95.0,4489,Male
4,OpenFace,LFW_gender,59.0,88.0,23.0,1511,Female
5,OpenFace,LFW_gender,55.0,85.0,12.0,4489,Male
6,DeepFace,LFW_gender,61.0,79.0,33.0,1511,Female
7,DeepFace,LFW_gender,65.0,76.0,44.0,4489,Male


In [14]:
# Write results to csv file
result_benchmark_df.to_csv("./result/deepface_benchmark_gender.csv", index=False)

### Etc testing